In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pytesseract


In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!apt-get install tesseract-ocr


In [ ]:
!pip install pytesseract


In [ ]:
import cv2
import pytesseract
import re
from transformers import pipeline

# Initialize the summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Path to the video file
video_path = "/content/drive/MyDrive/Intern/London Underground Extravaganza All 11 Lines! 29 November 2016.mp4"

# Open the video file
cap = cv2.VideoCapture(video_path)

# Check if the video file opened successfully
if not cap.isOpened():
    print("Error: Unable to open video file.")
    exit()

# Variable to store the text of the previous frame
prev_frame_text = ""
captions = []  # Store captions for story generation

# Loop through each frame in the video
while cap.isOpened():
    # Read the frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform OCR to extract text from the frame
    extracted_text = pytesseract.image_to_string(gray_frame)

    # Remove symbols from the extracted text
    extracted_text_cleaned = re.sub(r'[^\w\s]', '', extracted_text)

    # Check if the extracted text is the same as the text from the previous frame
    if extracted_text_cleaned == prev_frame_text:
        continue  # Skip storing captions for this frame

    # Add extracted caption to the list
    captions.append(extracted_text_cleaned)

    # Update the previous frame text
    prev_frame_text = extracted_text_cleaned

# Release the video capture object
cap.release()

# Generate story using captions
story_prompt = " ".join(captions)

# Initialize the ChatGPT pipeline
chatgpt = pipeline("text-generation", model="EleutherAI/gpt-neo-1.3B")

# Generate story using ChatGPT
story = chatgpt(story_prompt, max_length=1000, do_sample=False)[0]['generated_text']

# Print the generated story
print("Generated Story:")
print(story)
